In [25]:
#!pip3 install --upgrade pip
!pip install python-dotenv openai
!pip3 install pinecone-client

In [28]:
# To check your SDK version
!pip show pinecone-client

Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/it/Documents/prof_ratemyprofessor/.venv/lib/python3.12/site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: pinecone
Note: you may need to restart the kernel to use updated packages.


In [30]:
# To use the chat completion endpoint for Pinecone Assistant with the Python SDK
pip install --upgrade pinecone-client pinecone-plugin-assistant

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 5.0.1
    Uninstalling pinecone-client-5.0.1:
      Successfully uninstalled pinecone-client-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.0.1 requires pinecone-client==5.0.1, but you have pinecone-client 4.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Create requirements.txt

After installing the necessary libraries, you can use `pip-chill` to create a file called `requirements.txt` which helps the team install the required libraries to run this project on their local machine.

In [88]:
pip install pip-chill

Note: you may need to restart the kernel to use updated packages.


Run the following command to generate the `requirements.txt` file:

In [89]:
pip-chill >requirements.txt


Usage:   
  /Users/it/Documents/prof_ratemyprofessor/.venv/bin/python -m pip <command> [options]

no such option: -c
Note: you may need to restart the kernel to use updated packages.


Uncomment the following line to install the required libraries from the `requirements.txt` file:

In [ ]:
# pip install -r requirements.txt

In [58]:
from dotenv import load_dotenv
import os

from pinecone import Pinecone
from pinecone import ServerlessSpec

# load env file
load_dotenv()

True

In [80]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv('PINECODE_API_KEY'))
pc

When creating an index, import the `ServerlessSpec` class from the `pinecone` module.
If you created the index directly from Pinecone, you can skip this step.

In [81]:
pc.create_index(
  name="rmp-ai",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '3364179800b09bfaecefcae15bd66ed4', 'Date': 'Thu, 22 Aug 2024 12:04:42 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [82]:
import json
data = json.load(open('reviews.json'))
reviews = data['reviews']

## Embedding data to OpenAI
Here, we are going to convert our string data into a semantic vector representation. Basically, we are goingt to get some numerical representation of the text, which can be used for similarity searches. This is done using the Pinecone Text Embedding model.

In [83]:
from openai import OpenAI

# instantiate the OpenAI client
client = OpenAI()

processed_data = []

for review in reviews:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small", 
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [84]:
processed_data[0]

{'values': [-0.024080375,
  0.002041923,
  -0.014358231,
  0.0046292683,
  -0.014344595,
  0.0030236824,
  -0.027543804,
  0.02868919,
  -0.012183361,
  0.008631301,
  0.03935219,
  -0.0076290886,
  -0.030952692,
  -0.05901465,
  0.00026099288,
  0.006708689,
  0.02232139,
  0.03793409,
  0.029916389,
  0.05257867,
  0.021503257,
  0.0030134558,
  0.004520184,
  0.02060331,
  -0.058141973,
  -0.061250877,
  0.007901799,
  -0.009517612,
  0.0077109016,
  0.013621911,
  0.06626876,
  -0.032125346,
  -0.012360623,
  -0.033243462,
  -0.047669873,
  0.015626337,
  0.017426228,
  0.01649901,
  -0.006879133,
  0.0033373001,
  -0.0032810534,
  0.019676095,
  -0.025457567,
  -0.02994366,
  0.030979963,
  -0.0010968093,
  -0.05285138,
  0.0108675305,
  0.01359464,
  0.026807485,
  -0.020630583,
  0.031525385,
  0.05413312,
  0.026507502,
  -0.042488363,
  4.769778e-05,
  0.015040008,
  0.034797914,
  0.026248427,
  -0.0017879611,
  0.057051126,
  0.011051611,
  0.021653248,
  -0.009974402,
  0.0

## Vectorize data and index it in Pinecone

Here, we gonna upsert the processed data into the Pinecone index. This is similar to insert or update in a database.

In [85]:
index = pc.Index("rmp-ai")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 30}

To check the summary or description of our index. check if the `total_vector_count` corresponds to the number of reviews we have.

In [87]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}